In [1]:
import pandas as pd
import json
import copy

from csv_cleaner_helpers import *

# let's create the mother meta_df we need for taking care of the data read from almost anywhere 
meta_cols_in_variables = [
    "colname", "dtype", "varname", "col_exp", "col_exp_source", "units", "min", "max", "scale", 
    "decimal_places", "max_digits", "choices", "foreign_key", "foreign_key_related_name", "null_meaning", 
]

meta_variables = [
    "varname", "db_name", "main_desc", "main_desc_source", "notes", "cols",
    "section", "subsection"
]

df_variables = pd.DataFrame(columns=meta_variables)
df_cols_in_variables = pd.DataFrame(columns=meta_cols_in_variables)



with open('resulting_vars_dic_NEW.json') as json_file:
    crisisdb_data = json.load(json_file)

for variable, inner_data in crisisdb_data.items():
    # dic to be appended
    my_var_dic = {
        "varname": variable,
        "db_name": "crisisdb",
        "main_desc": inner_data["main_desc"],
        "main_desc_source": inner_data["main_desc_source"],
        "notes": inner_data["notes"],
        "cols": inner_data["cols"],
        "section": inner_data["section"],
        "subsection": inner_data["subsection"],
    }
    df_variables = df_variables.append(my_var_dic, ignore_index=True)
    df_variables = df_variables.where(pd.notnull(df_variables), None)
    
    number_of_cols = int(inner_data["cols"])
    for i in range(1, number_of_cols +1):
        column_key = "col" + str(i)
        my_col_dic = copy.deepcopy(inner_data[column_key])
        my_col_dic["colname"] = inner_data[column_key]["varname"]
        my_col_dic["varname"] = variable
        my_col_dic["null_meaning"]: inner_data["null_meaning"]
        my_col_dic["col_exp"]: inner_data[column_key]["var_exp"]
        my_col_dic["col_exp_source"]: inner_data[column_key]["var_exp_source"]
        del my_col_dic["var_exp_source"]
        del my_col_dic["var_exp"]
        df_cols_in_variables = df_cols_in_variables.append(my_col_dic, ignore_index=True)
        df_cols_in_variables = df_cols_in_variables.where(pd.notnull(df_cols_in_variables), None)



In [6]:
#main_majid_dic = convert_two_dfs_to_python_vars_dic(df_variables, df_cols_in_variables, my_db_name="general")
main_majid_df, main_majid_dff2 = convert_crisisdb_vars_dic_to_two_dfs('resulting_vars_dic_NEW.json')
main_majid_dic = convert_two_dfs_to_python_vars_dic(main_majid_df, main_majid_dff2, my_db_name="general")


In [7]:
from pprint import pprint
print(main_majid_dic)

{'agricultural_population': {'db_name': 'general', 'main_desc': 'No Explanations.', 'main_desc_source': '', 'notes': 'Notes for the Variable agricultural_population are missing!', 'cols': 1, 'section': 'Economy Variables', 'subsection': 'Productivity', 'needsSeshatCommon': None, 'col1': {'colname': 'agricultural_population', 'dtype': ['IntegerField', 'NumberInput'], 'varname': 'agricultural_population', 'units': 'People', 'scale': 1000}}, 'arable_land': {'db_name': 'general', 'main_desc': 'No Explanations.', 'main_desc_source': '', 'notes': 'Notes for the Variable arable_land are missing!', 'cols': 1, 'section': 'Economy Variables', 'subsection': 'Productivity', 'needsSeshatCommon': None, 'col1': {'colname': 'arable_land', 'dtype': ['IntegerField', 'NumberInput'], 'varname': 'arable_land', 'units': 'mu?', 'scale': 1000}}, 'arable_land_per_farmer': {'db_name': 'general', 'main_desc': 'No Explanations.', 'main_desc_source': '', 'notes': 'Notes for the Variable arable_land_per_farmer are 

In [ ]:
vars_dic = df_variables.to_dict('index')
cols_dic = df_cols_in_variables.to_dict('index')

In [ ]:
cols_dic

In [ ]:
final_vars_dic = {}
for index, main_var_dic in vars_dic.items():
    if main_var_dic["varname"] not in final_vars_dic.keys():
        final_vars_dic[main_var_dic["varname"]] = copy.deepcopy(main_var_dic)
        del final_vars_dic[main_var_dic["varname"]]["varname"]
        # lets also get the no of cols for iterating in the next round.
        no_of_cols = int(main_var_dic["cols"])
        #for i in range(1, no_of_cols + 1):
        #    column_key = "col" + str(i)
        done_cols = []
        hits = 0
        for col_index, col_dic in cols_dic.items():
            if col_dic['colname'] in done_cols:
                break
            if col_dic['varname'] == main_var_dic['varname']:
                hits = hits + 1 
                column_key = "col" + str(hits)
                final_vars_dic[main_var_dic["varname"]][column_key] = {}
                for col_key, col_val in col_dic.items():
                    if col_val:
                        final_vars_dic[main_var_dic["varname"]][column_key][col_key] = col_val
                done_cols.append(col_dic['colname'])
                print(done_cols)
                #final_vars_dic[varname][column_key] = copy.deepcopy(col_dic)
    else:
        continue

In [ ]:
final_vars_dic["military_expense"]

## Cleaning

I need to do all the cleaning here ans spit out files that will serve in two fronts:
* creating vars_dic: For this we mostly need the column names and the meta data.
    * This will in turn create all the models and views and forms related to them.
* creating SQL scripts to populate the database. So we need clean files for that. where we know exactly what the null values are, what we do with them in that particular file and everything.

In [ ]:
import csv
import pandas as pd
from csv_cleaner_helpers import *

# NO SOURCE for this data as of Sep 7th.

df_agricultural_data = pd.read_csv("Qing_Datasets/CrisisDB_agricultural_data.csv")
df_agricultural_data.head(2)

In [ ]:
df_agricultural_data.columns = df_agricultural_data.columns.str.replace(" ", "_")

df_agricultural_data.columns = df_agricultural_data.columns.str.lower()
# the followings can be more easily done via the csv file itself here in Jupyter envirinment.
df_agricultural_data.columns = df_agricultural_data.columns.str.replace("_\(catties_per_capita\)", "")
df_agricultural_data.columns = df_agricultural_data.columns.str.replace("_\(1000\)", "")
df_agricultural_data.columns = df_agricultural_data.columns.str.replace("_\(mu\)", "")
df_agricultural_data.columns = df_agricultural_data.columns.str.replace("_\(1000_mu\)", "")

df_agricultural_data.rename(columns = {'year':'year_from'}, inplace = True)

df_agricultural_data = add_polity_col(df_agricultural_data)
df_agricultural_data = add_year_to(df_agricultural_data)

### Save the `CLEAN` file
#### TO: `Qing_Datasets_CLEAN/`

In [ ]:
# create separate dfs based on models in Django (tentative)
file_name = "CrisisDB_agricultural_data"
agricultural_data_vars_dic = {}
for column in list(df_agricultural_data.columns):
    if column == "year_from" or column == "year_to" or column == "polity_id":
        continue
    df_key = "df_" + column
    df_value = df_agricultural_data[["polity_id", "year_from", "year_to", column]].copy()
    agricultural_data_vars_dic[df_key] = df_value
    # CLEAN_CSV file saving:
    my_address = "Qing_Datasets_CLEAN/" + str(df_key) + "_FROM_" + file_name + "_CLEAN.csv"
    df_value.to_csv(my_address, index = False)

### Add 
- `total_population`, (probably already there)
- `agricultural_population`,
- `arable_land`,
- `arable_land_per_farmer`,
- `gross_grain_shared_per_agricultural_population`,
- `net_grain_shared_per_agricultural_population`,
- `surplus`,

### To
- `resulting_vars_dic`

In [ ]:
# We don't have any explanations at the moment
agri_explanations = {
    # 'total_population', "No Explanations." (will be there through other files)
       'agricultural_population': "No Explanations.", 
        'arable_land': "No Explanations.", 
        'arable_land_per_farmer': "No Explanations.", 
       'gross_grain_shared_per_agricultural_population': "No Explanations.", 
       'net_grain_shared_per_agricultural_population': "No Explanations.",
        'surplus': "No Explanations.", 
}

my_ints_dic_AGRI = {}
for col, col_exp in agri_explanations.items():
    my_ints_dic_AGRI[col] = {}
    if col in ['agricultural_population']:
        my_ints_dic_AGRI[col][col] = {
            'var_exp': col_exp,
            'units': "People",
            'min': 0,
            'scale': 1000}
    elif col in ['arable_land',]:
        my_ints_dic_AGRI[col][col] = {
            'var_exp': col_exp,
            'units': "mu?",
            'scale': 1000,}
    elif col in ['arable_land_per_farmer',]:
        my_ints_dic_AGRI[col][col] = {
            'var_exp': col_exp,
            'units': "mu?",}
    elif col in ['gross_grain_shared_per_agricultural_population',
                 'net_grain_shared_per_agricultural_population', 'surplus']:
        my_ints_dic_AGRI[col][col] = {
            'var_exp': col_exp,
            'units': "(catties per capita)",}
    else:
        print("ERROR: ", col, " is not in the list of options.")

for df_key, df_value in agricultural_data_vars_dic.items():
    var_name = df_key[3:]
    df_var = df_value
    if var_name == "total_population":
        continue
    my_ints_dic = my_ints_dic_AGRI[var_name]
    vars_dic_entry_maker(var_name, df_var,
                        my_ints_dic, {}, {}, {},
                         "Economy Variables", "Productivity",
                         notes = "Notes on the Variable",
                         main_desc = agri_explanations.get(var_name, "No Main Descriptions Provided."),
                        )
my_ints_dic_AGRI.keys()
#df_value

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

- Is there any way we can give this exact thing as the input the vars_dic.
- At the end of the day, we want to create the vars_dic based on these individual files.
- So we can potentially (and actually more practically) provide the data: columns and metadata and units and everything and let Python create the vars_dic for us. Isn't that cool.
- we only need to make sure we feed all the information to a mediator function between `our files` and our `Django code`. 

#### What does `vars_dic` need?

I believe we can use the data in both `ABC.csv` file and `ABC_metadata.csv` for this:

- A list of column names.
- The name of the main column (or the name of the variable)
- number of columns in the variable.
- section name
- subsection name
- description for each column
- main description for the model (variable)
- datatype for the column: MYINT, MYDEC etc.
- units
- What null values mean

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
import csv
import pandas as pd
from csv_cleaner_helpers import *

# Source??

df_milexpenses = pd.read_csv("Qing_Datasets/qing_milexpense.csv", delimiter="|")
df_milexpenses.head(15)

We will be dealing with two types of notes:
* One note is the type we have here which is a `note_meta` for the whole variable and this will go into the variablehierarchy notes.
* A secind type is `note_specific` related to a specific row of data. Something we don't have here.

In [ ]:
df_milexpenses.columns = df_milexpenses.columns.str.lower()

In [ ]:
df_milexpenses[["year_from", "year_to"]] = df_milexpenses.apply(period_to_year, separator= "-", 
                                                                col_name = "period" , axis=1)

In [ ]:
del df_milexpenses["period"]
del df_milexpenses["note"]

In [ ]:
df_milexpenses = add_polity_col(df_milexpenses)
df_milexpenses = move_col_to(df_milexpenses, "year_from", 1)
df_milexpenses = move_col_to(df_milexpenses, "year_to", 2)

df_milexpenses.expenditure = df_milexpenses.expenditure.astype("float")

In [ ]:
df_milexpenses.head(2)

In [ ]:
df_milexpenses

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_military_expense_CLEAN.csv"
df_milexpenses.to_csv(my_address, index = False)

### Add `disease_outbreak` to `resulting_vars_dic`

In [ ]:
# for each variable we need a set of four dictionaries
my_floats_dic = {
    'expenditure': {
        'var_exp': "The military expenses in millions silver taels.",
        'units': "millions silver taels",
                },
}

my_chars_dic = {
    'conflict': {
        'var_exp': "The name of the conflict",
                },
}

vars_dic_entry_maker("military_expense", df_milexpenses,
                     {}, my_floats_dic, my_chars_dic, {},
                    "Economy Variables", "State Finances",
                     notes = "Not sure about Section and Subsection.",
                    main_desc_source = "https://en.wikipedia.org/wiki/Disease_outbreak",
                     null_meaning = "The value is not available.")

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

# source has no Zotero link yet.
source= "李隆生《清代的国际贸易：白银流入、货币危机和晚清工业化》，台湾秀威科技股份有效公司2010年版，第148-155页。"

df_silver_inflow = pd.read_csv("Qing_Datasets/qing_silver_inflow.csv", delimiter="|")
df_silver_inflow.head(3)

In [ ]:
df_silver_inflow = df_silver_inflow.dropna().reset_index(drop=True)
df_silver_inflow.rename(columns= {"year": "year_from"}, inplace=True)

In [ ]:
df_silver_inflow = add_polity_col(df_silver_inflow)
df_silver_inflow = add_year_to(df_silver_inflow)

In [ ]:
# I will make two models out of these
# The numbers associated with be integers
# The units will be million taels
df_silver_inflow.head(3)

### Save the `CLEAN` file

In [ ]:
# create separate dfs based on models in Django (tentative)
silver_data_dic = {}
for column in list(df_silver_inflow.columns):
    if column == "year_from" or column == "year_to" or column == "polity_id":
        continue
    df_key = "df_" + column
    df_value = df_silver_inflow[["polity_id", "year_from", "year_to", column]].copy()
    silver_data_dic[df_key] = df_value
    # CLEAN_CSV file saving:
    my_address = "Qing_Datasets_CLEAN/" + str(df_key) + "_CLEAN.csv"
    df_value.to_csv(my_address, index = False)

### Add 
- `silver_inflow`,
- `silver_stock`

### TO:
- `resulting_vars_dic`

In [ ]:
silver_explanations = {
    "silver_inflow": "Silver inflow in Millions of silver taels??",
    "silver_stock": "Silver stock in Millions of silver taels??",
}

my_ints_dic_silver = {}
for col, col_exp in silver_explanations.items():
    my_ints_dic_silver[col] = {}
    my_ints_dic_silver[col][col] = {
        'var_exp': col_exp,
        'units': "Millions of silver taels??",
        'scale': 1000000,
    }

for df_key, df_value in silver_data_dic.items():
    var_name = df_key[3:]
    df_var = df_value
    my_ints_dic = my_ints_dic_silver[var_name]
    vars_dic_entry_maker(var_name, df_var,
                        my_ints_dic, {}, {}, {},
                         "Economy Variables", "State Finances",
                         notes = "Needs suoervision on the units and scale.",
                         main_desc = silver_explanations[var_name],
                        )
my_ints_dic.keys()
#df_value

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

# Total Population

In [ ]:
from csv_cleaner_helpers import *

In [ ]:
df_China_total_population = pd.read_csv("Qing_Datasets/ClioInfra_China_TotalPopulation.csv", delimiter="|")
df_China_total_population.head(3)

In [ ]:
#df_China_toal_population = df_China_toal_population.dropna(axis=0)
df_China_total_population = df_China_total_population.dropna().reset_index(drop=True)

In [ ]:
del df_China_total_population["Country Code"]
del df_China_total_population["Indicator"]
del df_China_total_population["Country Name"]

df_China_total_population.rename(columns= {"Year": "year_from",
                                          "Data": "total_population"}, inplace=True)

In [ ]:
df_China_total_population["polity_id"] = df_China_total_population.apply(polity_id_decider_qing, axis=1)
# bring polity_id col to the front of the df:
backup = df_China_total_population["polity_id"]
df_China_total_population.drop(labels=["polity_id"], axis=1,inplace = True)
df_China_total_population.insert(0, 'polity_id', backup)

In [ ]:
df_China_total_population.total_population = df_China_total_population.total_population.astype("int")

In [ ]:
df_China_total_population["year_to"] = df_China_total_population["year_from"]
backup = df_China_total_population["year_to"]
df_China_total_population.drop(labels=["year_to"], axis=1,inplace = True)
df_China_total_population.insert(2, 'year_to', backup)

In [ ]:
df_China_total_population.tail(3)

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_total_population_CLEAN.csv"
df_China_total_population.to_csv(my_address, index = False)

In [ ]:
# This is to replace all NaNs with Nones
# df_China_toal_population = df_China_toal_population.where(pd.notnull(df_China_toal_population), None)

### Add `total_population` to `resulting_vars_dic`

In [ ]:
my_ints_dic = {
    'total_population': {
        'var_exp': "The total population of a country (or a polity).",
        'units': "People",
        'min': 0,
        'scale': 1000,
                },}

vars_dic_entry_maker("total_population", df_China_total_population,
                    my_ints_dic,{},{},{},
                    "Social Complexity Variables", "Social Scale",
                    notes = "Note that the population values are scaled.",
                    main_desc = "Total population or simply population, of a given area is the total number of people in that area at a given time.",
                    null_meaning = "The value is not available.")

In [ ]:
# This is already in Zotero: 975BEGKF
df_China_toal_population_meta = pd.read_csv("Qing_Datasets/ClioInfra_China_TotalPopulation_metadata.csv", delimiter="|")
df_China_toal_population_meta

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

df_ClioInfra_China_GDPperCapita = pd.read_csv("Qing_Datasets/ClioInfra_China_GDPperCapita.csv", delimiter="|")
df_ClioInfra_China_GDPperCapita.head(4)

In [ ]:
df_ClioInfra_China_GDPperCapita = df_ClioInfra_China_GDPperCapita.dropna().reset_index(drop=True)

del df_ClioInfra_China_GDPperCapita["Country Code"]
del df_ClioInfra_China_GDPperCapita["Indicator"]
del df_ClioInfra_China_GDPperCapita["Country Name"]

df_ClioInfra_China_GDPperCapita.rename(columns= {"Year": "year_from",
                                          "Data": "gdp_per_capita"}, inplace=True)

df_ClioInfra_China_GDPperCapita.gdp_per_capita = df_ClioInfra_China_GDPperCapita.gdp_per_capita.astype("float")
df_ClioInfra_China_GDPperCapita.year_from = df_ClioInfra_China_GDPperCapita.year_from.astype("int")

In [ ]:
df_ClioInfra_China_GDPperCapita = add_polity_col(df_ClioInfra_China_GDPperCapita)
df_ClioInfra_China_GDPperCapita = add_year_to(df_ClioInfra_China_GDPperCapita)

In [ ]:
df_ClioInfra_China_GDPperCapita.head(4)

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_gdp_per_capita_CLEAN.csv"
df_ClioInfra_China_GDPperCapita.to_csv(my_address, index = False)

### Add `gdp_per_capita` to `resulting_vars_dic`

In [ ]:
my_floats_dic = {
    'gdp_per_capita': {
        'var_exp': "The Gross Domestic Product per capita, or GDP per capita, is a measure of a country's economic output that accounts for its number of people. It divides the country's gross domestic product by its total population.",
        'var_exp_source': "https://www.thebalance.com/gdp-per-capita-formula-u-s-compared-to-highest-and-lowest-3305848",
        'units': "Dollars (in 2009?)",
                },}

vars_dic_entry_maker("gdp_per_capita", df_ClioInfra_China_GDPperCapita,
                    {},my_floats_dic,{},{},
                    "Economy Variables", "Productivity",
                    notes = "The exact year based on which the value of Dollar is taken into account is not clear.",
                    main_desc = "The Gross Domestic Product per capita, or GDP per capita, is a measure of a country's economic output that accounts for its number of people. It divides the country's gross domestic product by its total population.",
                    main_desc_source = "https://www.thebalance.com/gdp-per-capita-formula-u-s-compared-to-highest-and-lowest-3305848",
                     null_meaning = "The value is not available.")

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

# the third one is the one we are using: MM6AEU7H
sources = ["https://www.ncei.noaa.gov/pub/data/paleo/historical/asia/china/reaches2020drought.txt",
           "https://www.ncei.noaa.gov/pub/data/paleo/historical/asia/china/reaches2020drought-severe.txt",
          "REACHES Chinese Historical Climate Database Qing Dynasty Drought Series https://www1.ncdc.noaa.gov/pub/data/paleo/historical/asia/china/reaches2020drought-category-sites.txt"]

df_REACHES_climate_data = pd.read_csv("Qing_Datasets/REACHES_climate_data.csv", delimiter="|")
df_REACHES_climate_data.head(2)

In [ ]:
df_REACHES_climate_data.columns = df_REACHES_climate_data.columns.str.lower()
df_REACHES_climate_data.rename(columns = {'year':'year_from',
                                         'socioec.turmoil':'socioeconomic_turmoil_event',
                                         'famine': 'famine_event',
                                         'crop.failure': "crop_failure_event",
                                         'locust': 'locust_event',
                                         'drought': "drought_event"}, inplace = True)
df_REACHES_climate_data.columns = df_REACHES_climate_data.columns.str.replace(".", "_")
df_REACHES_climate_data = df_REACHES_climate_data.astype({'year_from':'int',
                                                        'drought_event': 'int',
                                                          'locust_event': 'int',
                                                          'crop_failure_event': 'int',
                                                          'socioeconomic_turmoil_event': 'int',
                                                          'famine_event': 'int',})
del df_REACHES_climate_data["country"]
del df_REACHES_climate_data["source"]

In [ ]:
df_REACHES_climate_data = add_polity_col(df_REACHES_climate_data)
df_REACHES_climate_data = add_year_to(df_REACHES_climate_data)

In [ ]:
df_REACHES_climate_data.head(2)

### Save the `CLEAN` file
#### TO: `Qing_Datasets_CLEAN/`

In [ ]:
# create separate dfs based on models in Django (tentative)
REACHES_climate_data_dic = {}
for column in list(df_REACHES_climate_data.columns):
    if column == "year_from" or column == "year_to" or column == "polity_id":
        continue
    df_key = "df_" + column
    df_value = df_REACHES_climate_data[["polity_id", "year_from", "year_to", column]].copy()
    REACHES_climate_data_dic[df_key] = df_value
    # CLEAN_CSV file saving:
    my_address = "Qing_Datasets_CLEAN/" + str(df_key) + "_CLEAN.csv"
    df_value.to_csv(my_address, index = False)

### Add 
- `famine_events`,
- `crop_failure_events`,
- `socioeconomic_turmoil_events`,
- `locust_events`,
- `drought_events`

### To
- `resulting_vars_dic`

In [ ]:
events_explanations = {
    "famine_event": "number of geographic sites indicating famine",
    "crop_failure_event": "number of geographic sites indicating crop failure",
    "socioeconomic_turmoil_event": "number of geographic sites indicating socioeconomic turmoil",
    "locust_event": "number of geographic sites indicating locusts",
    "drought_event": "number of geographic sites indicating drought",
}

my_ints_dic_REACHES = {}
for col, col_exp in events_explanations.items():
    my_ints_dic_REACHES[col] = {}
    my_ints_dic_REACHES[col][col] = {
        'var_exp': col_exp,
        'units': "Numbers",
        'min': 0,}

for df_key, df_value in REACHES_climate_data_dic.items():
    var_name = df_key[3:]
    df_var = df_value
    my_ints_dic = my_ints_dic_REACHES[var_name]
    vars_dic_entry_maker(var_name, df_var,
                        my_ints_dic, {}, {}, {},
                         "Well Being", "Biological Well-Being",
                         notes = "Notes on the Variable",
                         main_desc = events_explanations[var_name],
                        main_desc_source = "https://www1.ncdc.noaa.gov/pub/data/paleo/historical/asia/china/reaches2020drought-category-sites.txt")
my_ints_dic.keys()
#df_value

<hr style="margin-top:0px;height:18px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

df_qing_disease = pd.read_csv("Qing_Datasets/qing_disease.csv", delimiter="|")
df_qing_disease.head(2)

In [ ]:
del df_qing_disease["Event"]
del df_qing_disease["Variable"]
del df_qing_disease["Main.Category"]
del df_qing_disease["Dataset"]

In [ ]:
df_qing_disease.rename(columns = {'PolID':'polity_id', 
                                  'date.from':'year_from',
                                 'date.to': 'year_to',
                                 'long': 'longitude',
                                 'lat': 'latitude',
                                 'Sub.Category': 'sub_category',
                                 'Magnitude': 'magnitude',
                                 'Duration': 'duration',}, inplace = True)

In [ ]:
df_qing_disease.head(2)

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_disease_outbreak_CLEAN.csv"
df_qing_disease.to_csv(my_address, index = False)

### Add `disease_outbreak` to `resulting_vars_dic`

In [ ]:
# for each variable we need a set of four dictionaries
my_floats_dic = {
    'longitude': {
        'var_exp': "The longitude (in degrees) of the place where the disease was spread.",
        'units': "Degrees",
        'min': -180,
        'max': 180,
                },
    'latitude': {
        'var_exp': "The latitude (in degrees) of the place where the disease was spread.",
        'units': "Degrees",
        'min': -180,
        'max': 180,
                },
    'elevation': {
        'var_exp': "Elevation from mean sea level (in meters) of the place where the disease was spread.",
        'units': "Meters",
        'min': 0,
        'max': 5000,
                },
}

my_text_selects_dic = {
    'sub_category': {
        'var_exp': "The category of the disease.",
                },
    'magnitude': {
        'var_exp': "How heavy the disease was.",
                },
    'duration': {
        'var_exp': "How long the disease lasted.",
                },
}

vars_dic_entry_maker("disease_outbreak", df_qing_disease,
                     {}, my_floats_dic, {}, my_text_selects_dic,
                    "Well Being", "Biological Well-Being",
                    main_desc = "A sudden increase in occurrences of a disease when cases are in excess of normal expectancy for the location or season.",
                    main_desc_source = "https://en.wikipedia.org/wiki/Disease_outbreak",
                     null_meaning = "The value is not available.")

<hr style="margin-top:0px;height:45px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

## COnflicts New Models

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

df_internal_conflicts = pd.read_csv("Qing_Datasets/internal_conflict.csv", delimiter="|")
df_internal_conflicts.head(2)

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_internal_conflict_CLEAN.csv"
df_internal_conflicts.to_csv(my_address, index = False)

### Add `internal_conflict` to `resulting_vars_dic`

In [ ]:
my_floats_dic = {
    'expenditure': {
        'var_exp': "The military expenses in millions silver taels.",
        'units': "millions silver taels",
                },
}

my_ints_dic = {
    'casualty': {
    'var_exp': "The number of people who died in this conflict.",
    'units': "People",
            },
}
my_chars_dic = {
    'conflict': {
        'var_exp': "The name of the conflict",
                },
    'leader': {
        'var_exp': "The leader of the conflict",
                },
}

vars_dic_entry_maker("internal_conflict", df_internal_conflicts,
                     my_ints_dic, my_floats_dic, my_chars_dic, {},
                    "Conflict Variables", "Internal Conflicts Subsection",
                     notes = "This is a new model definition fror internal conflicts",
                    main_desc_source = "",
                     null_meaning = "The value is not available.")

<hr style="margin-top:0px;height:45px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

df_external_conflict_sides = pd.read_csv("Qing_Datasets/external_conflict_side.csv", delimiter="|")
df_external_conflict_sides.head(2)

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_external_conflict_side_CLEAN.csv"
df_external_conflict_sides.to_csv(my_address, index = False)

### Add `internal_conflict` to `resulting_vars_dic`

In [ ]:
my_floats_dic = {
    'expenditure': {
        'var_exp': "The military expenses (from this side) in silver taels.",
        'units': "silver taels",
                },
}

my_ints_dic = {
    'casualty': {
    'var_exp': "The number of people who died (from this side) in this conflict.",
    'units': "People",
            },
}
my_chars_dic = {
    'conflict_name': {
        'var_exp': "The unique name of the conflict",
                },
    'leader': {
        'var_exp': "The leader of this side of conflict",
                },
}

my_foreign_keys_dic = {
    'conflict_side': {
        'var_exp': "The external_conflict which is the actual conflict we are talking about",
        'foreign_key': "External_conflict",
        'foreign_key_related_name': "External_conflicts",
    }
}

vars_dic_entry_maker("external_conflict_side", df_external_conflict_sides,
                     my_ints_dic, my_floats_dic, my_chars_dic, {}, my_foreign_keys_dic,
                    "Conflict Variables", "External Conflicts Subsection",
                     notes = "This is a new model definition for External conflict sides",
                    main_desc_source = "",
                     null_meaning = "The value is not available.")

<hr style="margin-top:0px;height:45px;background-image: linear-gradient(to right, red,orange,green,blue,indigo,violet);">

In [ ]:
from csv_cleaner_helpers import *
import pandas as pd

df_external_conflict = pd.read_csv("Qing_Datasets/external_conflict.csv", delimiter="|")
df_external_conflict.head(2)

### Save the `CLEAN` file

In [ ]:
my_address = "Qing_Datasets_CLEAN/df_external_conflict_CLEAN.csv"
df_external_conflict.to_csv(my_address, index = False)

### Add `internal_conflict` to `resulting_vars_dic`

In [ ]:
my_chars_dic = {
    'conflict_name': {
        'var_exp': "The unique name of this external conflict",
                },
}

# my_foreign_keys_dic = {
#     'side_'
# }
# I have not taken care of the foreign keys here, 
# but here I definitely need it:
# This might need to be done when the sides are already there in the db.

vars_dic_entry_maker_with_foreign_key("external_conflict", df_external_conflict,
                     {}, {}, my_chars_dic, {},
                    "Conflict Variables", "External Conflicts Subsection",
                     notes = "This is a new model definition fror External conflict sides",
                    main_desc_source = "",
                     null_meaning = "The value is not available.")

In [ ]:
dict_1 = {'John': 15, 'Rick': 10, 'Misa' : 12 }
dict_2 = {'Bonnie': 18,'Rick': 20,'Matt' : 16 }
dict_3 = {**dict_1,**dict_2}
print(dict_3)